In [73]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import random

# Requests sends and receives HTTP requests
import requests
from bs4 import BeautifulSoup

#used if i need wildcard in bs
import re

import time
import pprint

In [91]:
client = MongoClient('localhost', 27017)
db = client['scraper']
whole_page = db['whole_page']
budgets = db['budgets']


In [100]:
main_url= 'https://www.the-numbers.com/movie/budgets/all'
pages = list(np.arange(101, 6101, 100))
links = [main_url]
for page in pages:
    link = f'{main_url}/{page}'
    links.append(link)

In [96]:
# single request test
print(links[1])
r = requests.get(links[1])
r.status_code

https://www.the-numbers.com/movie/budgets/all/all/101


404

In [12]:
pprint.pprint(r.text)

('<!DOCTYPE html>\r\n'
 '<html xmlns:og="http://ogp.me/ns#" '
 'xmlns:fb="http://www.facebook.com/2008/fbml">\r\n'
 '<head>\r\n'
 '<!-- Global site tag (gtag.js) - Google Analytics -->\r\n'
 '<script async '
 'src="https://www.googletagmanager.com/gtag/js?id=UA-1343128-1"></script>\r\n'
 '<script>\r\n'
 '  window.dataLayer = window.dataLayer || [];\r\n'
 '  function gtag(){dataLayer.push(arguments);}\r\n'
 "  gtag('js', new Date());\r\n"
 '\r\n'
 "  gtag('config', 'UA-1343128-1');\r\n"
 '</script>\r\n'
 '<meta http-equiv="PICS-Label" content=\'(PICS-1.1 '
 '"https://www.icra.org/ratingsv02.html" l gen true for '
 '"https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) '
 '"https://www.rsac.org/ratingsv01.html" l gen true for '
 '"https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))\'>\r\n'
 '<!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" '
 '>-->\r\n'
 '<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n'
 '<meta name="format-detectio

In [14]:
#adding info to whole_page db of url and time to get as much info as possible

whole_page.insert_one({'html': r.content, 'url':links[0], 'time_scraped':time.time()})


In [16]:
soup = BeautifulSoup(r.text, 'html.parser')

In [42]:
test = soup.find_all('tr')[1]

In [43]:
print(test)

<tr><td class="data">1</td>
<td><a href="/box-office-chart/daily/2019/04/23">Apr 23, 2019</a></td>
<td><b><a href="/movie/Avengers-Endgame-(2019)#tab=summary">Avengers: Endgame</a></b></td>
<td class="data"> $400,000,000</td>
<td class="data"> $858,373,000</td>
<td class="data"> $2,797,800,564</td>
</tr>


In [50]:
data = test.find_all('td')
date = data[1].text
name = data[2].text
budget = data[3].text
domestic_gross = data[4].text
world_gross = data[5].text

In [59]:
print(data)

[<td class="data">1</td>, <td><a href="/box-office-chart/daily/2019/04/23">Apr 23, 2019</a></td>, <td><b><a href="/movie/Avengers-Endgame-(2019)#tab=summary">Avengers: Endgame</a></b></td>, <td class="data"> $400,000,000</td>, <td class="data"> $858,373,000</td>, <td class="data"> $2,797,800,564</td>]


In [52]:
budget_samp = soup.find_all('td', class_='data')[1]
# name = soup.find_all('td', {'href' : re.compile('/movie*')})

In [71]:
date, name, budget, domestic, world = [i.text for i in data[1:]]

In [72]:
print(date, name, budget, domestic, world)

Jul 20, 2017 Valerian and the City of a Thousand Planets  $180,000,000  $40,479,370  $215,098,356


In [69]:
# Test loop
rows = soup.find_all('tr')[1]
test = []
for idx in range(1,len(soup.find_all('tr'))):
    row = soup.find_all('tr')[idx]
    data = row.find_all('td')
    date, name, budget, domestic, world = [i.text for i in data[1:]]
    test.append([date, name, budget])
    

100


In [76]:
print(len(test))

[['Apr 23, 2019', 'Avengers: Endgame', '\xa0$400,000,000'], ['May 20, 2011', 'Pirates of the Caribbean: On Stranger Tides', '\xa0$379,000,000'], ['Apr 22, 2015', 'Avengers: Age of Ultron', '\xa0$365,000,000'], ['Dec 16, 2015', 'Star Wars Ep. VII: The Force Awakens', '\xa0$306,000,000'], ['Apr 25, 2018', 'Avengers: Infinity War', '\xa0$300,000,000'], ['May 24, 2007', 'Pirates of the Caribbean: At Worldâ\x80\x99s End', '\xa0$300,000,000'], ['Nov 13, 2017', 'Justice League', '\xa0$300,000,000'], ['Oct 6, 2015', 'Spectre', '\xa0$300,000,000'], ['Dec 18, 2019', 'Star Wars: The Rise of Skywalker', '\xa0$275,000,000'], ['May 23, 2018', 'Solo: A Star Wars Story', '\xa0$275,000,000'], ['Mar 7, 2012', 'John Carter', '\xa0$263,700,000'], ['Mar 23, 2016', 'Batman v Superman: Dawn of Justice', '\xa0$263,000,000'], ['Jul 11, 2019', 'The Lion King', '\xa0$260,000,000'], ['Nov 24, 2010', 'Tangled', '\xa0$260,000,000'], ['May 4, 2007', 'Spider-Man 3', '\xa0$258,000,000'], ['Apr 22, 2016', 'Captain Amer

In [101]:
print(links)

['https://www.the-numbers.com/movie/budgets/all', 'https://www.the-numbers.com/movie/budgets/all/101', 'https://www.the-numbers.com/movie/budgets/all/201', 'https://www.the-numbers.com/movie/budgets/all/301', 'https://www.the-numbers.com/movie/budgets/all/401', 'https://www.the-numbers.com/movie/budgets/all/501', 'https://www.the-numbers.com/movie/budgets/all/601', 'https://www.the-numbers.com/movie/budgets/all/701', 'https://www.the-numbers.com/movie/budgets/all/801', 'https://www.the-numbers.com/movie/budgets/all/901', 'https://www.the-numbers.com/movie/budgets/all/1001', 'https://www.the-numbers.com/movie/budgets/all/1101', 'https://www.the-numbers.com/movie/budgets/all/1201', 'https://www.the-numbers.com/movie/budgets/all/1301', 'https://www.the-numbers.com/movie/budgets/all/1401', 'https://www.the-numbers.com/movie/budgets/all/1501', 'https://www.the-numbers.com/movie/budgets/all/1601', 'https://www.the-numbers.com/movie/budgets/all/1701', 'https://www.the-numbers.com/movie/budget

In [103]:
# Requests
main_url= 'https://www.the-numbers.com/movie/budgets/all'
pages = list(np.arange(101, 6101, 100))
links = [main_url]
for page in pages:
    link = f'{main_url}/{page}'
    links.append(link)

# Loop for all pages     
for link in links:   
    print(link)
    r = requests.get(link)
    r.status_code
    
    whole_page.insert_one({'html': r.content, 'url':links[0], 'time_scraped':time.time()})
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # Parser  
    rows = soup.find_all('tr')[1]
    budget_list = []
    for idx in range(1,len(soup.find_all('tr'))):
        row = soup.find_all('tr')[idx]
        data = row.find_all('td')
        date, name, budget, domestic, world = [i.text for i in data[1:]]
        budget_list.append([date, name, budget, domestic, world])
        budgets.insert_one({'date':date, 'title':name, 'budget':budget, 'domestic_gross':domestic, 'worldwide_gross':world})
    
    # Mongo print for sanity
    print(len(budget_list))
    print(db.budgets.find().count())
    
    # Add time before next request 
    slp = random.randint(5,10)
    time.sleep(slp)

https://www.the-numbers.com/movie/budgets/all
100
100


/Users/Michael/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


https://www.the-numbers.com/movie/budgets/all/101
100
200
https://www.the-numbers.com/movie/budgets/all/201
100
300
https://www.the-numbers.com/movie/budgets/all/301
100
400
https://www.the-numbers.com/movie/budgets/all/401
100
500
https://www.the-numbers.com/movie/budgets/all/501
100
600
https://www.the-numbers.com/movie/budgets/all/601
100
700
https://www.the-numbers.com/movie/budgets/all/701
100
800
https://www.the-numbers.com/movie/budgets/all/801
100
900
https://www.the-numbers.com/movie/budgets/all/901
100
1000
https://www.the-numbers.com/movie/budgets/all/1001
100
1100
https://www.the-numbers.com/movie/budgets/all/1101
100
1200
https://www.the-numbers.com/movie/budgets/all/1201
100
1300
https://www.the-numbers.com/movie/budgets/all/1301
100
1400
https://www.the-numbers.com/movie/budgets/all/1401
100
1500
https://www.the-numbers.com/movie/budgets/all/1501
100
1600
https://www.the-numbers.com/movie/budgets/all/1601
100
1700
https://www.the-numbers.com/movie/budgets/all/1701
100
18

In [78]:
print(type(db))

<class 'pymongo.database.Database'>


In [104]:
budgets_pd = pd.DataFrame(list(budgets.find()))

In [105]:
budgets_pd.head()

,_id,date,title,budget,domestic_gross,worldwide_gross
0,5f17b39c8fbba2e9aaf4294b,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,5f17b39c8fbba2e9aaf4294c,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875"
2,5f17b39c8fbba2e9aaf4294d,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,396,099,202"
3,5f17b39c8fbba2e9aaf4294e,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,065,478,084"
4,5f17b39c8fbba2e9aaf4294f,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754"


In [106]:
budgets_pd.tail()

,_id,date,title,budget,domestic_gross,worldwide_gross
6038,5f17b5918fbba2e9aaf4411d,Unknown,Red 11,"$7,000",$0,$0
6039,5f17b5918fbba2e9aaf4411e,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
6040,5f17b5918fbba2e9aaf4411f,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
6041,5f17b5918fbba2e9aaf44120,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0
6042,5f17b5918fbba2e9aaf44121,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041"


In [107]:
budgets_pd.to_csv('budgets_pd.csv', index=False)

In [ ]:
# How to convert to zip file from PD dataframe

# compression_opts = dict(method='zip', archive_name='out.csv')  
# df.to_csv('out.zip', index=False, compression=compression_opts)